In [1]:
import os
import argparse
from collections import defaultdict
import dlib
import cv2
from sklearn.externals import joblib
import numpy as np
import pandas as pd
from pipeline.eyecolor import extract_eyecolor_features
from process_data import load_facedetector
from pipeline.face_utils import *
from pipeline import pipeline_utils
from pipeline.texture import extract_features_eyes,extract_features_faceborder,extract_features_mouth,extract_features_nose
import glob


# img = cv2.imread("aa.jpeg")
# img = cv2.imread("190.png")
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
face_detector, sp68 = load_facedetector()
# face_crop_list, landmarks_list = get_crops_landmarks(face_detector, sp68, img)
# print(landmarks_list)
import dlib.cuda as cuda
print(cuda.get_num_devices())
print(dlib.DLIB_USE_CUDA)
# cv2.imshow("aaa",face_crop_list[0])
# cv2.show()
# cv2.waitKey()

/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


2
True


In [2]:

final_score_clf = 0.0
final_score_HSV = 0.0
final_feature_vector = None
final_valid_seg = False
scale = 768
def extract_visual_artifact(img):
    face_crop_list, landmarks_list = get_crops_landmarks(face_detector, sp68, img)
    scale = 768

    if (len(face_crop_list) == 1):

        face_crop = face_crop_list[0]
        landmarks = landmarks_list[0].copy()

        out_size = pipeline_utils.new_size(face_crop.shape[1], face_crop.shape[0], large_dim=scale)
        scale_x = float(out_size[1]) / face_crop.shape[1]
        scale_y = float(out_size[0]) / face_crop.shape[0]

        landmarks_resize = landmarks.copy()
        landmarks_resize[:, 0] = landmarks_resize[:, 0] * scale_x
        landmarks_resize[:, 1] = landmarks_resize[:, 1] * scale_y

        face_crop_resize = cv2.resize(face_crop, (int(out_size[1]), int(out_size[0])), interpolation=cv2.INTER_LINEAR)

        feature_eyecolor, distance_HSV, valid_seg = extract_eyecolor_features(landmarks_resize, face_crop_resize)
        features_eyes = extract_features_eyes(landmarks, face_crop, scale=scale)
        features_mounth = extract_features_mouth(landmarks, face_crop, scale=scale)
        features_nose = extract_features_nose(landmarks, face_crop, scale=scale)
        features_face = extract_features_faceborder(landmarks, face_crop, scale=scale)
        feature = np.concatenate([feature_eyecolor, features_eyes, features_mounth, features_nose, features_face], axis=0)
    else:
        feature = np.array([0])
#     print(feature)
    return feature

In [9]:

# df_path = '/hdd/tam/df_in_the_wild/image/train_sub/1_df'
# real_path = '/hdd/tam/df_in_the_wild/image/train_sub/0_real'
number_iter = 1000
df_path = '/hdd/tam/UADFV/image/test/1_df'
# df_list = os.listdir(df_path)
features = []
labels = []
cont = 0
for vid_path in glob.glob(df_path+"/*.jpg"):
    # vid_path = os.path.join(df_path, vid_name)
#     print(vid_path)

#     try:
    img = cv2.imread(vid_path)
#     print("re re   :  ",vid_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    feature = extract_visual_artifact(img)
#     print(feature)
    if len(feature) < 3:
        print("aaaa")
        continue
    # print(feature)
    features.append(feature)
    labels.append([1])
    cont+=1
    print(cont)
    if cont == number_iter:
        break
#     except:
#         continue

real_path = '/hdd/tam/UADFV/image/test/0_real'
cont = 0
for vid_path in glob.glob(real_path+"/*.jpg"):
    # vid_path = os.path.join(real_path, vid_name)
#     print(vid_path)

#     try:
    img = cv2.imread(vid_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    feature = extract_visual_artifact(img)
#         print(feature)
    if len(feature) < 3:
        continue
    features.append(feature)
    labels.append([0])
    cont+=1
    print(cont)
    if cont == number_iter:
        break
#     except:
#         continue


1
2
3
4
5
6
7
8
aaaa
9
aaaa
10
aaaa
11
12
13
14
aaaa
15
16
17
18
19
aaaa
20
21
aaaa
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
aaaa
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
aaaa
65
66
67
68
69
70
71
72
73
74
75
76
77
78
aaaa
79
80
81
aaaa
82
83
84
85
aaaa
86
87
88
89
90
91
92
93
94
95
96
97
aaaa
98
99
100
101
102
103
104
105
106
107
108
109
aaaa
110
111
112
aaaa
113
114
115
116
117
118
119
aaaa
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
aaaa
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
aaaa
172
173
174
175
176
177
178
179
180
181
182
183
184
aaaa
185
186
187
188
189
190
191
192
193
194
aaaa
195
196
197
198
199
200
aaaa
201
aaaa
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
aaaa
246
247
aaaa
248
2

961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000


In [10]:
import pickle
info_dict = {'features':features,'labels':labels}
# print(info_dict)
with open("test_uadfv.pkl", 'wb') as f:
    pickle.dump(info_dict, f)

In [12]:
labels[1000:]

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]

In [25]:
info_dict = None
with open("test_dftimit.pkl", 'rb') as f:
    info_dict = pickle.load(f)

In [13]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


CLF_NAMES = ["mlp", "logreg"]
CLFS = [
    MLPClassifier(alpha=0.1, hidden_layer_sizes=(64, 64, 64), learning_rate_init=0.001, max_iter=300),
    LogisticRegression(),
]

import pickle
info_dict = None
with open("train_uadfv.pkl", 'rb') as f:
    info_dict = pickle.load(f)

features= info_dict['features']
labels = info_dict['labels']
features = np.array(features)
with open("test_uadfv.pkl", 'rb') as f:
    info_dict = pickle.load(f)

features2= info_dict['features']
labels2 = info_dict['labels']
# print(features)
features2 = np.array(features2)
# print(features.shape)
for name, clf in zip(CLF_NAMES, CLFS):
    clf.fit(features, labels)
#     print(name)
    print(clf.score(features2,labels2))
    # joblib.dump(clf, os.path.join(output_path, name + '.pkl'))

/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.643
0.7085


/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [5]:
print(labels)

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1],